In [1]:
import cv2
from PIL import Image 
import time

def to_pil(cv_img):
    img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img)
import numpy as np

In [ ]:
np.random.randn()

In [83]:
a = np.random.randn(2)
db = np.random.randn(10, 2)
db /= np.linalg.norm(db, axis=1).reshape(-1, 1)
a /= np.linalg.norm(a)

In [100]:
%precision 5
cos = np.dot(db, db[0])
cos

array([ 1.     , -0.7835 , -0.93192,  0.8478 ,  0.16645,  0.57783,
       -0.98654, -0.54647, -0.96586,  0.99467])

In [99]:
%precision 5
cosa0 = np.dot(a, db[0])
cosa0

0.8956173401976478

In [101]:
%precision 5
cosa0 = np.dot(a, db[1])
cosa0

-0.4253064641088211

In [98]:
%precision 5
cosa = np.dot(db, a)
cosa

array([ 0.89562, -0.42531, -0.67331,  0.5234 ,  0.5877 ,  0.88056,
       -0.81081, -0.1169 , -0.7498 ,  0.845  ])

In [85]:
%precision 5
theta = np.arccos(cos[1:])
theta

array([2.47108, 2.77046, 0.55898, 1.40357, 0.95473, 2.97731, 2.14894,
       2.87953, 0.10325])

In [95]:
theta - thetaa0

array([ 2.0101 ,  2.30948,  0.098  ,  0.94259,  0.49375,  2.51633,
        1.68796,  2.41855, -0.35773])

In [96]:
thetaa0 - theta

array([-2.0101 , -2.30948, -0.098  , -0.94259, -0.49375, -2.51633,
       -1.68796, -2.41855,  0.35773])

In [55]:
start = time.time()
for _ in range(100):
    np.dot(db, a)
(time.time() - start) / 100

0.02272399663925171

In [18]:
a[:, 2][a[:, 2] > 0.5] = 1
a[:, 2][a[:, 2] <= 0.5] = 0

In [22]:
a[a[:,2] > 0][:, :2] > (0.5, 0.3)

array([[ True,  True],
       [False,  True],
       [False,  True],
       [ True, False]])

In [4]:
a[1] = 0

In [9]:
import sys

In [20]:
f = open('/home/allen/two_people.jpg', 'rb')

In [21]:
img_str = f.read()

In [13]:
import io
from PIL import Image

In [23]:
byte = io.BytesIO(img_str)
image = Image.open(byte)

In [27]:
import torch
torch.backends.cudnn.enabled

True

In [28]:
torch.contiguous_format

torch.contiguous_format